In [1]:
import tensorflow as tf

from tensorflow.python.framework import ops

ops.reset_default_graph()

E:\Anaconda3_5_0_0\lib\site-packages\h5py\__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
feature_clum = []
# feature_clum.append(tf.feature_column.numeric_column(key="image", shape=[28*28]))
for i in range(28*28):
    feature_clum.append(tf.feature_column.numeric_column(key=( "pixel%d" % i )))

In [3]:
runConfig = tf.estimator.RunConfig(model_dir="./model")

digit_class = tf.estimator.DNNClassifier(feature_columns=feature_clum,
                                         hidden_units=[200,200],n_classes = 10,config=runConfig)

INFO:tensorflow:Using config: {'_model_dir': './model', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': None, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x000002CAFA65F1D0>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [4]:
import pandas as pd

CSV_COLUMN_NAMES = ['label']

for i in range(28*28):
    CSV_COLUMN_NAMES.append("pixel%d" % i)

In [5]:

def load_csv(path,label=None):
    minist = pd.read_csv(path,names=CSV_COLUMN_NAMES,header=0)
    return minist

In [6]:
import numpy as np

In [7]:
def slip_train_data():
    minist = load_csv("../digit-recognizer/train.csv")
    df = minist.sample(frac=0.8) # 全部打乱
    cut_idx = int(round(0.1 * df.shape[0]))
    df_test, df_train = df.iloc[:cut_idx], df.iloc[cut_idx:]
    train_x,train_y = df_train,df_train.pop('label')
    test_x,test_y = df_test,df_test.pop('label')
    return (train_x,train_y),(test_x,test_y)


In [8]:
(train_x,train_y),(test_x,test_y) = slip_train_data()

In [9]:
tf.logging.set_verbosity(tf.logging.INFO)

In [10]:
def input_fn_train(features,labels,batch_size):
    dataset = tf.data.Dataset.from_tensor_slices((dict(features),labels))
    dataset = dataset.shuffle(1000).batch(batch_size)
    return dataset

In [11]:
digit_class.train(input_fn=lambda:input_fn_train(train_x,train_y,128),steps=10000)

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into ./model\model.ckpt.
INFO:tensorflow:loss = 13377.64, step = 0
INFO:tensorflow:global_step/sec: 9.33292
INFO:tensorflow:loss = 210.54776, step = 100 (10.717 sec)
INFO:tensorflow:global_step/sec: 8.6964
INFO:tensorflow:loss = 221.03206, step = 200 (11.524 sec)
INFO:tensorflow:Saving checkpoints for 237 into ./model\model.ckpt.
INFO:tensorflow:Loss for final step: 60.79298.


In [41]:
def input_fn_eval(features,labels,batch_size):
    features = dict(features)
    if labels is None:
        inputs = features
    else:
        inputs = (features,labels)
    dataset = tf.data.Dataset.from_tensor_slices(inputs)
    dataset = dataset.shuffle(1000).batch(batch_size)
    return dataset

eval_result = digit_class.evaluate(input_fn=lambda:input_fn_eval(test_x,test_y,128))
print(eval_result)

INFO:tensorflow:Calling model_fn.
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel0', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel1', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel10', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel100', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel101', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel102', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel103', shape=(1,), d

DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel149', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel15', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel150', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel151', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel152', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel153', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel154', shape=(1,), default_value=None, dtype=tf.fl

DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel248', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel249', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel25', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel250', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel251', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel252', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel253', shape=(1,), default_value=None, dtype=tf.fl

DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel299', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel3', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel30', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel300', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel301', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel302', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel303', shape=(1,), default_value=None, dtype=tf.floa

DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel349', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel35', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel350', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel351', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel352', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel353', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel354', shape=(1,), default_value=None, dtype=tf.fl

DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel4', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel40', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel400', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel401', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel402', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel403', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel404', shape=(1,), default_value=None, dtype=tf.floa

DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel499', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel5', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel50', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel500', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel501', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel502', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel503', shape=(1,), default_value=None, dtype=tf.floa

DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel599', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel6', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel60', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel600', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel601', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel602', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel603', shape=(1,), default_value=None, dtype=tf.floa

DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel698', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel699', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel7', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel70', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel700', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel701', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel702', shape=(1,), default_value=None, dtype=tf.floa

DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel92', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel93', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel94', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel95', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel96', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel97', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None).
DEBUG:tensorflow:Transforming feature_column _NumericColumn(key='pixel98', shape=(1,), default_value=None, dtype=tf.float32,

In [34]:
CSV_COLUMN_NAMES_PRE = []

for i in range(28*28):
    CSV_COLUMN_NAMES_PRE.append("pixel%d" % i)
pre_cvs = pd.read_csv("../digit-recognizer/test.csv",names=CSV_COLUMN_NAMES_PRE,header=0)[0:1]
expected = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
predictions = digit_class.predict(input_fn=lambda:input_fn_eval(pre_cvs,labels=None,batch_size=128))
# print(predictions)
for pred_dict,expec in zip(predictions,expected):
    template = ('\nPrediction is "{}" ({:.1f}%), expected "{}"')
    class_id = pred_dict['class_ids'][0]
    probability = pred_dict['probabilities'][class_id]
    
    print(template.format(expected[class_id],
                              100 * probability, expec))

INFO:tensorflow:Calling model_fn.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Restoring parameters from ./model\model.ckpt-474
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.

Prediction is "9" (12.0%), expected "0"
